In [30]:
import httpx
import asyncio
import datetime as dt
import pylint
from lxml import etree
import time
import re

In [31]:
OLDEST = dt.datetime(1958, 8, 4)

In [ ]:
def to_saturday(date, scalar=1):
    # rounds the date to a Saturday
    while date.weekday() != 5:
        date = date + scalar*dt.timedelta(days=1)
    return date

In [ ]:
def date_generator(latest_date, delta=1):
    # infintite generator for dates that takes the timedelta as a paramater
    latest_date = to_saturday(latest_date)
    curr = latest_date
    while True:
        yield curr
        curr -= dt.timedelta(weeks=delta)

In [ ]:
dates = date_generator(round_date(dt.datetime.today()))
tasks = [await single_page(client, date) for date in dates]

pages = await asyncio.gather(*tasks)



NameError: name 'client' is not defined

In [ ]:
# testing parser feed() and read_events() methods
async def single_page(client, date):
    url = get_url(date)
    parser = etree.HTMLParser(target=BillboardTarget())
    main_frame = []
    async with client.stream('GET', URL) as response:
        async for chunk in response.aiter_text():
            parser.feed(chunk)
            for content in parser.read_events():
                main_frame += incrementally_build_frame(content)
    return date, main_frame
                

In [116]:
class BillboardTarget:
    VALID_TAG = 'ul'
    VALID_CLASS = 'o-chart-results-list-row //'
    def __init__(self):
        self.content = []
        self.capture = False
        self.curr_tag = None
        self.curr_data = ''

    def start(self, tag, attrib):
        if tag == self.VALID_TAG and self.VALID_CLASS in attrib['class']:
            self.capture = True
            self.curr_tag = tag
            self.curr_data = ''

    def end(self, tag):
        if self.capture and tag == self.curr_tag:
            self.content.append(self.curr_data)
            self.capture = False
            self.curr_tag = None

    def data(self, text):
        if self.capture and not text.isspace():
            text = re.sub(r"NEW", "", text)
            text = re.sub(r"RE-\nENTRY", "", text)
            self.curr_data += text.strip() + '_'

    def close(self):
        return self.content


In [117]:
test_doc = open('Billboard-page.html')
parser = etree.HTMLParser(target=BillboardTarget())
e = etree.parse(test_doc, parser)
test_doc.close()
e

['1_Roses Are Red (My Love)_Bobby Vinton_1_1_7_1_1_7_',
 '2_The Wah Watusi_The Orlons_4_2_7_4_2_7_',
 "3_I Can_'_t Stop Loving You_Ray Charles_3_1_12_3_1_12_",
 '4_The Stripper_David Rose and His Orchestra_2_1_11_2_1_11_',
 '5_Sealed With A Kiss_Brian Hyland_5_5_7_5_5_7_',
 '6_Wolverton Mountain_Claude King_7_6_9_7_6_9_',
 '7_Johnny Get Angry_Joanie Sommers_11_7_9_11_7_9_',
 '8_Speedy Gonzales_Pat Boone_13_8_6_13_8_6_',
 '9_Gravy (For My Mashed Potatoes)_Dee Dee Sharp_9_9_6_9_9_6_',
 '10_Palisades Park_Freddy Cannon_6_3_11_6_3_11_',
 "11_Al Di La_'_Emilio Pericoli_10_6_10_10_6_10_",
 '12_Ahab, The Arab_Ray Stevens_18_12_4_18_12_4_',
 '13_Breaking Up Is Hard To Do_Neil Sedaka_19_13_4_19_13_4_',
 "14_It Keeps Right On A-Hurtin_'_Johnny Tillotson_8_3_11_8_3_11_",
 "15_Dancin_'_Party_Chubby Checker_20_15_5_20_15_5_",
 "16_I_'_ll Never Dance Again_Bobby Rydell_14_14_8_14_14_8_",
 '17_Having A Party_Sam Cooke_17_17_9_17_17_9_',
 '18_Theme From Dr. Kildare (Three Stars Will Shine Tonight)_Ric

In [118]:
e = [re.sub(r"_\'_", "'", row) for row in e]
e
# listy = [row.split('_')[i] for row in e for i in [0, 1, 2, 5]]
# listy

['1_Roses Are Red (My Love)_Bobby Vinton_1_1_7_1_1_7_',
 '2_The Wah Watusi_The Orlons_4_2_7_4_2_7_',
 "3_I Can't Stop Loving You_Ray Charles_3_1_12_3_1_12_",
 '4_The Stripper_David Rose and His Orchestra_2_1_11_2_1_11_',
 '5_Sealed With A Kiss_Brian Hyland_5_5_7_5_5_7_',
 '6_Wolverton Mountain_Claude King_7_6_9_7_6_9_',
 '7_Johnny Get Angry_Joanie Sommers_11_7_9_11_7_9_',
 '8_Speedy Gonzales_Pat Boone_13_8_6_13_8_6_',
 '9_Gravy (For My Mashed Potatoes)_Dee Dee Sharp_9_9_6_9_9_6_',
 '10_Palisades Park_Freddy Cannon_6_3_11_6_3_11_',
 "11_Al Di La'Emilio Pericoli_10_6_10_10_6_10_",
 '12_Ahab, The Arab_Ray Stevens_18_12_4_18_12_4_',
 '13_Breaking Up Is Hard To Do_Neil Sedaka_19_13_4_19_13_4_',
 "14_It Keeps Right On A-Hurtin'Johnny Tillotson_8_3_11_8_3_11_",
 "15_Dancin'Party_Chubby Checker_20_15_5_20_15_5_",
 "16_I'll Never Dance Again_Bobby Rydell_14_14_8_14_14_8_",
 '17_Having A Party_Sam Cooke_17_17_9_17_17_9_',
 '18_Theme From Dr. Kildare (Three Stars Will Shine Tonight)_Richard Chamb

In [ ]:
def incrementally_build_frame(e):
# look ahead uppercase, lookbehind lowercase, not space
    # pat = re.compile(r"^_+(?P<position>\d+)_+(?P<song>[^_]+)_+(?P<artist>[^_]+)(?:_+\d+){2}_+(?P<wks_on_chart>\d+)")
    columns = ['position', 'song', 'artist', 'wks_on_chart']
    cleaned_e = [re.sub(r"_'_", "'", entry) for entry in e]
    frame = [re.search(pat, row).groupdict() for row in cleaned_e if re.search(pat, row)]
    return frame

    


[{'position': '1',
  'song': 'Roses Are Red (My Love)',
  'artist': 'Bobby Vinton',
  'wks_on_chart': '7'},
 {'position': '2',
  'song': 'The Wah Watusi',
  'artist': 'The Orlons',
  'wks_on_chart': '7'},
 {'position': '3',
  'song': "I Can't Stop Loving You",
  'artist': 'Ray Charles',
  'wks_on_chart': '12'},
 {'position': '4',
  'song': 'The Stripper',
  'artist': 'David Rose and His Orchestra',
  'wks_on_chart': '11'},
 {'position': '5',
  'song': 'Sealed With A Kiss',
  'artist': 'Brian Hyland',
  'wks_on_chart': '7'},
 {'position': '6',
  'song': 'Wolverton Mountain',
  'artist': 'Claude King',
  'wks_on_chart': '9'},
 {'position': '7',
  'song': 'Johnny Get Angry',
  'artist': 'Joanie Sommers',
  'wks_on_chart': '9'},
 {'position': '8',
  'song': 'Speedy Gonzales',
  'artist': 'Pat Boone',
  'wks_on_chart': '6'},
 {'position': '9',
  'song': 'Gravy (For My Mashed Potatoes)',
  'artist': 'Dee Dee Sharp',
  'wks_on_chart': '6'},
 {'position': '10',
  'song': 'Palisades Park',
  'a